# RDS

Nesta seção fazemos o armazenamento do dataset original e alterado no AWS RDS.

Utilizamos as seguintes bibliotecas:
- boto3: para interagir com os serviços da AWS.
- pymysql: para conectar-se a bancos de dados MySQL.
- pandas: para manipulação de dados.
- dotenv: para carregar variáveis de ambiente
- pathlib: para manipulação de caminhos de sistema de arquivos

Precisamos adicionar os custom modules, os scripts, ao env path, para podermos importá-los.

In [15]:
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0])

if module_path not in sys.path:
    sys.path.append(module_path)

## Queries

Podemos verificar se tudo está funcionando corretamente listando as informações do nosso RDS instance.

In [16]:
import boto3

# Inicializa uma seção usando AWS RDS
rds_client = boto3.client('rds')

response = rds_client.describe_db_instances()

# Print das informações das DB instances
for db_instance in response['DBInstances']:
    print(f"DB Instance Identifier: {db_instance['DBInstanceIdentifier']}")
    print(f"DB Instance Class: {db_instance['DBInstanceClass']}")
    print(f"DB Engine: {db_instance['Engine']}")
    print(f"DB Instance Status: {db_instance['DBInstanceStatus']}")
    #print(f"DB Endpoint: {db_instance['Endpoint']['Address']}")
    #print(f"DB Port: {db_instance['Endpoint']['Port']}")
    #print(f"DB Instance ARN: {db_instance['DBInstanceArn']}")
    print("-" * 60)

DB Instance Identifier: database-project-sprint5
DB Instance Class: db.t3.micro
DB Engine: mysql
DB Instance Status: available
------------------------------------------------------------


Vamos criar a database para o dataset original se ainda não existir. Da mesma forma criaremos a table se a mesma não existir.

Com os dados do .env fazemos a conexão com o BD.

In [17]:
import pymysql
from dotenv import load_dotenv
import os

load_dotenv()

connection = pymysql.connect(
    host=os.getenv('RDS_HOST'),
    user= os.getenv('RDS_USER'),
    password=os.getenv('RDS_PASSWORD'),
    port=int(os.getenv('RDS_PORT')),
    database=os.getenv('DB_NAME')
)

Criamos o DATABASE se ele ainda não existir.

In [18]:
cursor = connection.cursor()

cursor.execute(f"CREATE DATABASE IF NOT EXISTS {os.getenv('DB_NAME')}")

1

Podemos agora importar o dataset para um pandas dataframe.

In [19]:
import pandas as pd

df = pd.read_csv('../data/HotelReservations.csv')
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


Importamos o script para criar a nossa query CREATE TABLE relacionada com o dataset original se ela ainda não existe. Podemos ver o resultado.

In [20]:
from scripts import query_create_table

dtype_mapping = {
    'object': 'VARCHAR(255)',
    'int64': 'INT',
    'float64': 'FLOAT',
    'bool': 'BOOLEAN',
    'datetime64[ns]': 'TIMESTAMP'
}

table_name = 'reservations'
create_table_query = query_create_table.generate(table_name, df, dtype_mapping)
print(create_table_query)

CREATE TABLE IF NOT EXISTS reservations (
    Booking_ID VARCHAR(255),
    no_of_adults INT,
    no_of_children INT,
    no_of_weekend_nights INT,
    no_of_week_nights INT,
    type_of_meal_plan VARCHAR(255),
    required_car_parking_space INT,
    room_type_reserved VARCHAR(255),
    lead_time INT,
    arrival_year INT,
    arrival_month INT,
    arrival_date INT,
    market_segment_type VARCHAR(255),
    repeated_guest INT,
    no_of_previous_cancellations INT,
    no_of_previous_bookings_not_canceled INT,
    avg_price_per_room FLOAT,
    no_of_special_requests INT,
    booking_status VARCHAR(255)
);


Executamos a query.

In [21]:
cursor.execute(create_table_query)

0

Vamos fazer o mesmo para o dataset alterado. Primeiro precisamos mapear os valores desejados na descrição do projeto e dropamos a coluna avg_price_per_room.

In [22]:
df_label = df.copy()

def classify_price(price):
    if price <= 85:
        return 1
    elif 85 < price < 115:
        return 2
    else:
        return 3
    
df_label['label_avg_price_per_room'] = df_label['avg_price_per_room'].apply(classify_price)

df_label.drop(columns=['avg_price_per_room'], axis=1, inplace=True)
df_label.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,booking_status,label_avg_price_per_room
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,0,Not_Canceled,1
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,1,Not_Canceled,2
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,0,Canceled,1
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,0,Canceled,2
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,0,Canceled,2


Importamos o script para criar a nossa query CREATE TABLE relacionada com o dataset alterado dessa vez.

In [23]:
from scripts import query_create_table


table_name_labeled = 'labeled_reservations'
create_table_query = query_create_table.generate(table_name_labeled, df_label, dtype_mapping)
print(create_table_query)

CREATE TABLE IF NOT EXISTS labeled_reservations (
    Booking_ID VARCHAR(255),
    no_of_adults INT,
    no_of_children INT,
    no_of_weekend_nights INT,
    no_of_week_nights INT,
    type_of_meal_plan VARCHAR(255),
    required_car_parking_space INT,
    room_type_reserved VARCHAR(255),
    lead_time INT,
    arrival_year INT,
    arrival_month INT,
    arrival_date INT,
    market_segment_type VARCHAR(255),
    repeated_guest INT,
    no_of_previous_cancellations INT,
    no_of_previous_bookings_not_canceled INT,
    no_of_special_requests INT,
    booking_status VARCHAR(255),
    label_avg_price_per_room INT
);


Executamos a ultima query criada e fazemos os commits de todas as alterações. Depois fechamos a conexão com o banco de dados para proseguir com o upload dos dados.

In [24]:
cursor.execute(create_table_query)
connection.commit()

cursor.close()
connection.close()

## Salvando os datasets no AWS RDS

Após criadas as duas tables podemos de fato exportar os dados para o RDS.

In [25]:
from scripts import aws_rds

aws_rds.upload(df,table_name)

Os dados foram exportados com sucesso.


Fazemos o mesmo para o dataset alterado.

In [26]:
aws_rds.upload(df_label,table_name_labeled)

Os dados foram exportados com sucesso.


Por fim podemos testar se os dados foram enviados corretamente fazendo o import deles como a seguir.

In [27]:
df_final = aws_rds.download(table_name_labeled)
df_final

Os dados foram importados com sucesso.


,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,booking_status,label_avg_price_per_room
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,0,Not_Canceled,1
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,1,Not_Canceled,2
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,0,Canceled,1
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,0,Canceled,2
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,0,Canceled,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,1,Not_Canceled,3
36271,INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,2,Canceled,2
36272,INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,2,Not_Canceled,2
36273,INN36274,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,0,Canceled,2


Podemos concluir salvando esse nosso dataframe final para um csv para utilizarmos em outras seções.

In [28]:
df_final.to_csv('../data/HotelReservationsLabel.csv', index=False)